In [ ]:
def affine_encrypt(a, b, x, m):
    
    y = (a * x + b) % m
    
    return y

def affine_decrypt(a_inv, b, y, m):
    
    x = a_inv * (y - b) % m
    
    return x

In [ ]:
def create_frequency(ciphertext, m=2):
    # Calculate the digram frequencies in the ciphertext
    n = len(ciphertext)
    digram_counts = {}
    for i in range(n - m + 1):
        digram = ciphertext[i:i+m]
        if digram in digram_counts:
            digram_counts[digram] += 1
        else:
            digram_counts[digram] = 1
    # Rank the digram_counts dictionary by frequency
    sorted_digrams = sorted(digram_counts.items(), key=lambda x: x[1], reverse=True)
    return sorted_digrams

In [ ]:
def create_letter_frequency(text):
    n = len(text)
    #freq = vector(ZZ, 26)
    MS = MatrixSpace(ZZ, 26, 2)
    freq = MS()
    
    for i in range(26):
        freq[i,0] = i   # first column of the row stores the alphabet letter itself
    
    for i in range(n):
        index = ord(text[i]) - ord('a')
        freq[index,1] = freq[index,1] + 1    # second column stores the frequency
    
    
    return freq

def bubble_sort(lst):
    n = len(lst.rows())
    ordered_list = lst
    
    #print(str(ordered_list))
    
    for i in range(n):
        for j in [0 .. n-i-2]:
            if ordered_list[j,1] < ordered_list[j + 1,1]:
                temp = ordered_list[j]
                ordered_list[j] = ordered_list[j + 1]
                ordered_list[j + 1] = temp
    #print(str(ordered_list))
    
    return ordered_list

def next_max_idx(arr, curr_max_idx):
    new_max_idx = 0
    temp_min_val = 0
    
    for i in range(len(arr)):
        if arr[i] > temp_min_val and (curr_max_idx == 26 or arr[i] < arr[curr_max_idx]):
            new_max_idx = i
            temp_min_val = arr[new_max_idx]
    
    return new_max_idx

def decrypt_message(y, a_inv, b, m):
    
    x = ""
    
    for i in range(len(y)):
        y_num = ord(y[i]) - ord('a')     # ord('a') = 97
        x_num = affine_decrypt(a_inv, b, y_num, m)
        x     = x + chr(int(x_num) + ord('a'))

    return x
        
def inv(a):
    
    m = 26
    Zm = Integers(m) # Here we give ourselves shorthand for the modular integers
    n = Zm(a) # Here we ask for a as an element of that ring
    n = n^(-1)
    
    return n

def solve(x1_1, x1_2, x2_1, x2_2, y1, y2, y3, y4, m):
    
    Zm=Integers(m) # Mod 26
    MS_Zm1 = MatrixSpace(Zm, 2, 2) # '2, 2' refer to number of rows and columns of the matrices represented by this space
    MS_Zm2 = MatrixSpace(Zm, 2, 2)
    MS_Key = MatrixSpace(Zm, 2, 2)
    
    
    A_zm1 = MS_Zm1([x1_1,x1_2,x2_1,x2_2])  # we solve for a and b in a*x1+b=y1; a*x2+b=y2
    #A_zm2 = MS_Zm2([x2_1,1,x2_2,1])
    print("a_zm1", A_zm1)
    #print("a_zm2", A_zm2)
    Y_zm1 = vector(Zm, [y1, y3])
    Y_zm2 = vector(Zm, [y2, y4])
    print("y_zm1", Y_zm1)
    print("y_zm2", Y_zm2)
    
    X_zm1 = A_zm1 \ Y_zm1
    X_zm2 = A_zm1 \ Y_zm2
    print("x_zm1", X_zm1)
    print("x_zm2", X_zm2)
    
    print("first val: ", X_zm1[0] )
    
    #key = MS_Key()
    return X_zm1

In [ ]:
# Below is the frequency-analysis attack on the affine cipher

m = 26
y = "LMQETXYEAGTXCTUIEWNCTXLZEWUAISPZYVAPEWLMGQWYAXFTCJMSQCADAGTXLMDXNXSNPJQSYVAPRIQSMHNOCVAXFV".lower()
x = ""
digram_freq = [x.lower() for x in ["TH", "HE", "IN", "ER", "AN", "RE", "ED", "ON", "ES", "ST"]]
freq = create_frequency(y)
#print(freq)
#freq = bubble_sort(freq)   # one array will be used to pick the first highest frequency ciphertext letter and the other for the second one

prev_high_freq_ciphertext_lttr = -1


for i in range(len(digram_freq)):
    #Grab TH First
    #high_freq_plaintext_lttr = ord(letter_freq[i]) - ord('a')
    digram_pair = digram_freq[i]
    
    digram_high_freq_char1 = digram_pair[0]
    high_freq_plaintext_lttr1 = ord(digram_high_freq_char1) - ord('a')
    
    digram_high_freq_char2 = digram_pair[1]
    high_freq_plaintext_lttr2 = ord(digram_high_freq_char2) - ord('a')
    
    print(digram_pair)
    
    print(high_freq_plaintext_lttr1, high_freq_plaintext_lttr2)
    
    #print(high_freq_plaintext_lttr)
    
    for j in range(len(digram_freq)): # for j in [i+1 .. m-1]:
        
        if j != i:
            
            next_digram_pair = digram_freq[j]
    
            next_digram_high_freq_char1 = next_digram_pair[0]
            next_high_freq_plaintext_lttr1 = ord(next_digram_high_freq_char1) - ord('a')
    
            next_digram_high_freq_char2 = next_digram_pair[1]
            next_high_freq_plaintext_lttr2 = ord(next_digram_high_freq_char2) - ord('a')
            
            print(next_high_freq_plaintext_lttr1, next_high_freq_plaintext_lttr2) 
            
            
            
            print("plaintext higher freq two digrams are: {" + str(digram_freq[i]) + ", " + str(digram_freq[j]) + "}\n")

            high_freq_ciphertext_lttr = 0 # prev_high_freq_ciphertext_lttr + 1 # next_max_idx(freq, prev_high_freq_ciphertext_lttr) # return the idx of the next max in the list of freq. This is the Z_26 representation of the highest freq ciphertext letter
            
            while high_freq_ciphertext_lttr < 26 and freq[high_freq_ciphertext_lttr][1] > 0:  # if freq of the digram is > 0

                next_high_freq_ciphertext_lttr = 0 # high_freq_ciphertext_lttr + 1 # next_max_idx(freq, high_freq_ciphertext_lttr)    # second next max freq ciphertext letter

                while next_high_freq_ciphertext_lttr < 26 and freq[next_high_freq_ciphertext_lttr][1] > 0:

                    if next_high_freq_ciphertext_lttr != high_freq_ciphertext_lttr:
                        print("ciphertext higher freq two letters are: {" + freq[high_freq_ciphertext_lttr][0] + ", " + freq[next_high_freq_ciphertext_lttr][0] + "}\n")
                        
                        # unicode values of the cipher texts
                        y1 = ord(freq[high_freq_ciphertext_lttr][0][0]) - ord('a')
                        y2 = ord(freq[high_freq_ciphertext_lttr][0][1]) - ord('a')
                        y3 = ord(freq[next_high_freq_ciphertext_lttr][0][0]) - ord('a')
                        y4 = ord(freq[next_high_freq_ciphertext_lttr][0][1]) - ord('a')
                        
                        print("asde: ", high_freq_plaintext_lttr1, high_freq_plaintext_lttr2, next_high_freq_plaintext_lttr1, next_high_freq_plaintext_lttr2)
                        print("asdf: ", y1,y2,y3,y4)
                        
                        
                        solution = solve(high_freq_plaintext_lttr1, high_freq_plaintext_lttr2, next_high_freq_plaintext_lttr1, next_high_freq_plaintext_lttr2, y1,y2,y3,y4, m)
                        
                        print(solution)
                        
                        if gcd(solution[0], m) == 1:
                            a_inv = inv(solution[0])
                            b = solution[1]
                            x = decrypt_message(y, a_inv, b, m)                    
                            print("a_inv = " + str(a_inv) + ", b = " + str(b) + ", plaintext = " + x)
                            ch = input("Press Enter to continue or E to exit...")
                            if ch == "E" or ch == "e":
                                sys.exit("Exiting the program!!")
                        else:
                            print("a = " + str(solution[0]) + " is not a valid solution\n")

                    next_high_freq_ciphertext_lttr = next_high_freq_ciphertext_lttr + 1 # next_max_idx(freq, next_high_freq_ciphertext_lttr)

                high_freq_ciphertext_lttr = high_freq_ciphertext_lttr + 1 # next_max_idx(freq, high_freq_ciphertext_lttr)
            